In [3]:
import gc
gc.collect()
print("Data Cleared")

Data Cleared


In [4]:
import sys
import pandas as pd
import numpy as np
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
import random

from sklearn.preprocessing import MinMaxScaler

import implicit


In [527]:
# Load the data like we did before
raw_data = pd.read_csv('C:\\Users\Manu\\Documents\\Untitled Folder\\Orginial Dataset\\BOOKSPURCHHISTORY.csv')
data = raw_data.drop(['TIMESTAMP','SUBSTATE','WeekofYear'],axis=1)
data['Purch'] = 1 
# Drop NaN columns
data = data.dropna()

In [528]:
# The implicit library expects data as a item-user matrix so we
# create two matricies, one for fitting the model (item-user) 
# and one for recommendations (user-item)
sparse_item_user = sparse.csr_matrix((data['Purch'].astype(float), (data['BookID'], data['UserID'])))
sparse_user_item = sparse.csr_matrix((data['Purch'].astype(float), (data['UserID'], data['BookID'])))

# Initialize the als model and fit it using the sparse item-user matrix
model = implicit.als.AlternatingLeastSquares(factors=20, regularization=0.1, iterations=20)

# Calculate the confidence by multiplying it by our alpha value.
alpha_val = 15
data_conf = (sparse_item_user * alpha_val).astype('double')

#Fit the model
model.fit(data_conf)


100%|████████████████████████████████████████████████████████████████| 20.0/20 [00:02<00:00,  7.10it/s]


In [1]:
#---------------------
# FIND SIMILAR ITEMS
#---------------------

# Find the 10 most similar to Jay-Z
item_id = 140361 #Jay-Z
n_similar = 10

# Use implicit to get similar items.
similar = model.similar_items(item_id, n_similar)

# Print the names of our most similar artists
for item in similar:
    idx, score = item
    print(data.BookID.loc[data.BookID == idx].iloc[0])

NameError: name 'model' is not defined

In [534]:
data.head()

,BookID,UserID,Purch
0,140361,608502,1
1,118603,673204,1
2,170523,641650,1
3,111924,619531,1
4,178056,628661,1


In [538]:
#------------------------------
# CREATE USER RECOMMENDATIONS
#------------------------------

# Create recommendations for user with id 2025
user_id = 608502

# Use the implicit recommender.
recommended = model.recommend(user_id, sparse_user_item)

books = []
scores = []

# Get artist names from ids
for item in recommended:
    idx, score = item
    books.append(data.BookID.loc[data.BookID == idx].iloc[0])
    scores.append(score)
#'UserID':user_id,
# Create a dataframe of artist names and scores
recommendations = pd.DataFrame({'BookID': books, 'score': scores})

print(recommendations)

   BookID     score
0  159750  0.229144
1  138981  0.205793
2  106808  0.205593
3  164397  0.197822
4  175371  0.190236
5  105696  0.189423
6  118603  0.181385
7  175147  0.159942
8  148634  0.159567
9  120845  0.158918


In [542]:
#------------------------------
# CREATE USER RECOMMENDATIONS
#------------------------------

# Create recommendations for user with id 2025
#user_id = 608502


user = []
books = []
scores = []
i=0
for user_id in (data.UserID.unique()):
    # Use the implicit recommender.
    recommended = model.recommend(user_id, sparse_user_item,30)
       
    # Get artist names from ids
    for item in recommended:
        idx, score = item
        user.append(user_id)
        books.append(idx)
        scores.append(score)
        i+=1
        print(i)
        
# Create a dataframe of artist names and scores
recommendations = pd.DataFrame({'UserID': user,'BookID': books, 'score': scores})

print(recommendations)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060


5137
5138
5139
5140
5141
5142
5143
5144
5145
5146
5147
5148
5149
5150
5151
5152
5153
5154
5155
5156
5157
5158
5159
5160
5161
5162
5163
5164
5165
5166
5167
5168
5169
5170
5171
5172
5173
5174
5175
5176
5177
5178
5179
5180
5181
5182
5183
5184
5185
5186
5187
5188
5189
5190
5191
5192
5193
5194
5195
5196
5197
5198
5199
5200
5201
5202
5203
5204
5205
5206
5207
5208
5209
5210
5211
5212
5213
5214
5215
5216
5217
5218
5219
5220
5221
5222
5223
5224
5225
5226
5227
5228
5229
5230
5231
5232
5233
5234
5235
5236
5237
5238
5239
5240
5241
5242
5243
5244
5245
5246
5247
5248
5249
5250
5251
5252
5253
5254
5255
5256
5257
5258
5259
5260
5261
5262
5263
5264
5265
5266
5267
5268
5269
5270
5271
5272
5273
5274
5275
5276
5277
5278
5279
5280
5281
5282
5283
5284
5285
5286
5287
5288
5289
5290
5291
5292
5293
5294
5295
5296
5297
5298
5299
5300
5301
5302
5303
5304
5305
5306
5307
5308
5309
5310
5311
5312
5313
5314
5315
5316
5317
5318
5319
5320
5321
5322
5323
5324
5325
5326
5327
5328
5329
5330
5331
5332
5333
5334
5335
5336


8413
8414
8415
8416
8417
8418
8419
8420
8421
8422
8423
8424
8425
8426
8427
8428
8429
8430
8431
8432
8433
8434
8435
8436
8437
8438
8439
8440
8441
8442
8443
8444
8445
8446
8447
8448
8449
8450
8451
8452
8453
8454
8455
8456
8457
8458
8459
8460
8461
8462
8463
8464
8465
8466
8467
8468
8469
8470
8471
8472
8473
8474
8475
8476
8477
8478
8479
8480
8481
8482
8483
8484
8485
8486
8487
8488
8489
8490
8491
8492
8493
8494
8495
8496
8497
8498
8499
8500
8501
8502
8503
8504
8505
8506
8507
8508
8509
8510
8511
8512
8513
8514
8515
8516
8517
8518
8519
8520
8521
8522
8523
8524
8525
8526
8527
8528
8529
8530
8531
8532
8533
8534
8535
8536
8537
8538
8539
8540
8541
8542
8543
8544
8545
8546
8547
8548
8549
8550
8551
8552
8553
8554
8555
8556
8557
8558
8559
8560
8561
8562
8563
8564
8565
8566
8567
8568
8569
8570
8571
8572
8573
8574
8575
8576
8577
8578
8579
8580
8581
8582
8583
8584
8585
8586
8587
8588
8589
8590
8591
8592
8593
8594
8595
8596
8597
8598
8599
8600
8601
8602
8603
8604
8605
8606
8607
8608
8609
8610
8611
8612


10043
10044
10045
10046
10047
10048
10049
10050
10051
10052
10053
10054
10055
10056
10057
10058
10059
10060
10061
10062
10063
10064
10065
10066
10067
10068
10069
10070
10071
10072
10073
10074
10075
10076
10077
10078
10079
10080
10081
10082
10083
10084
10085
10086
10087
10088
10089
10090
10091
10092
10093
10094
10095
10096
10097
10098
10099
10100
10101
10102
10103
10104
10105
10106
10107
10108
10109
10110
10111
10112
10113
10114
10115
10116
10117
10118
10119
10120
10121
10122
10123
10124
10125
10126
10127
10128
10129
10130
10131
10132
10133
10134
10135
10136
10137
10138
10139
10140
10141
10142
10143
10144
10145
10146
10147
10148
10149
10150
10151
10152
10153
10154
10155
10156
10157
10158
10159
10160
10161
10162
10163
10164
10165
10166
10167
10168
10169
10170
10171
10172
10173
10174
10175
10176
10177
10178
10179
10180
10181
10182
10183
10184
10185
10186
10187
10188
10189
10190
10191
10192
10193
10194
10195
10196
10197
10198
10199
10200
10201
10202
10203
10204
10205
10206
10207
10208
1020

...

304709
304710
304711
304712
304713
304714
304715
304716
304717
304718
304719
304720
304721
304722
304723
304724
304725
304726
304727
304728
304729
304730
304731
304732
304733
304734
304735
304736
304737
304738
304739
304740
304741
304742
304743
304744
304745
304746
304747
304748
304749
304750
304751
304752
304753
304754
304755
304756
304757
304758
304759
304760
304761
304762
304763
304764
304765
304766
304767
304768
304769
304770
304771
304772
304773
304774
304775
304776
304777
304778
304779
304780
304781
304782
304783
304784
304785
304786
304787
304788
304789
304790
304791
304792
304793
304794
304795
304796
304797
304798
304799
304800
304801
304802
304803
304804
304805
304806
304807
304808
304809
304810
304811
304812
304813
304814
304815
304816
304817
304818
304819
304820
304821
304822
304823
304824
304825
304826
304827
304828
304829
304830
304831
304832
304833
304834
304835
304836
304837
304838
304839
304840
304841
304842
304843
304844
304845
304846
304847
304848
304849
304850
304851

310510
310511
310512
310513
310514
310515
310516
310517
310518
310519
310520
310521
310522
310523
310524
310525
310526
310527
310528
310529
310530
310531
310532
310533
310534
310535
310536
310537
310538
310539
310540
310541
310542
310543
310544
310545
310546
310547
310548
310549
310550
310551
310552
310553
310554
310555
310556
310557
310558
310559
310560
310561
310562
310563
310564
310565
310566
310567
310568
310569
310570
310571
310572
310573
310574
310575
310576
310577
310578
310579
310580
310581
310582
310583
310584
310585
310586
310587
310588
310589
310590
310591
310592
310593
310594
310595
310596
310597
310598
310599
310600
310601
310602
310603
310604
310605
310606
310607
310608
310609
310610
310611
310612
310613
310614
310615
310616
310617
310618
310619
310620
310621
310622
310623
310624
310625
310626
310627
310628
310629
310630
310631
310632
310633
310634
310635
310636
310637
310638
310639
310640
310641
310642
310643
310644
310645
310646
310647
310648
310649
310650
310651
310652

312850
312851
312852
312853
312854
312855
312856
312857
312858
312859
312860
312861
312862
312863
312864
312865
312866
312867
312868
312869
312870
312871
312872
312873
312874
312875
312876
312877
312878
312879
312880
312881
312882
312883
312884
312885
312886
312887
312888
312889
312890
312891
312892
312893
312894
312895
312896
312897
312898
312899
312900
312901
312902
312903
312904
312905
312906
312907
312908
312909
312910
312911
312912
312913
312914
312915
312916
312917
312918
312919
312920
312921
312922
312923
312924
312925
312926
312927
312928
312929
312930
312931
312932
312933
312934
312935
312936
312937
312938
312939
312940
312941
312942
312943
312944
312945
312946
312947
312948
312949
312950
312951
312952
312953
312954
312955
312956
312957
312958
312959
312960
312961
312962
312963
312964
312965
312966
312967
312968
312969
312970
312971
312972
312973
312974
312975
312976
312977
312978
312979
312980
312981
312982
312983
312984
312985
312986
312987
312988
312989
312990
312991
312992

319013
319014
319015
319016
319017
319018
319019
319020
319021
319022
319023
319024
319025
319026
319027
319028
319029
319030
319031
319032
319033
319034
319035
319036
319037
319038
319039
319040
319041
319042
319043
319044
319045
319046
319047
319048
319049
319050
319051
319052
319053
319054
319055
319056
319057
319058
319059
319060
319061
319062
319063
319064
319065
319066
319067
319068
319069
319070
319071
319072
319073
319074
319075
319076
319077
319078
319079
319080
319081
319082
319083
319084
319085
319086
319087
319088
319089
319090
319091
319092
319093
319094
319095
319096
319097
319098
319099
319100
319101
319102
319103
319104
319105
319106
319107
319108
319109
319110
319111
319112
319113
319114
319115
319116
319117
319118
319119
319120
319121
319122
319123
319124
319125
319126
319127
319128
319129
319130
319131
319132
319133
319134
319135
319136
319137
319138
319139
319140
319141
319142
319143
319144
319145
319146
319147
319148
319149
319150
319151
319152
319153
319154
319155

325710
325711
325712
325713
325714
325715
325716
325717
325718
325719
325720
325721
325722
325723
325724
325725
325726
325727
325728
325729
325730
325731
325732
325733
325734
325735
325736
325737
325738
325739
325740
325741
325742
325743
325744
325745
325746
325747
325748
325749
325750
325751
325752
325753
325754
325755
325756
325757
325758
325759
325760
325761
325762
325763
325764
325765
325766
325767
325768
325769
325770
325771
325772
325773
325774
325775
325776
325777
325778
325779
325780
325781
325782
325783
325784
325785
325786
325787
325788
325789
325790
325791
325792
325793
325794
325795
325796
325797
325798
325799
325800
325801
325802
325803
325804
325805
325806
325807
325808
325809
325810
325811
325812
325813
325814
325815
325816
325817
325818
325819
325820
325821
325822
325823
325824
325825
325826
325827
325828
325829
325830
325831
325832
325833
325834
325835
325836
325837
325838
325839
325840
325841
325842
325843
325844
325845
325846
325847
325848
325849
325850
325851
325852

In [558]:
recommendations.columns

Index(['UserID', 'BookID', 'score'], dtype='object')

In [570]:
data.columns = ['BookID','UserID','score']

In [571]:
Final_Data = pd.concat([data,recommendations],axis =0).reset_index(drop = True)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [572]:
Final_Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 369382 entries, 0 to 369381
Data columns (total 3 columns):
BookID    369382 non-null int64
UserID    369382 non-null int64
score     369382 non-null float64
dtypes: float64(1), int64(2)
memory usage: 8.5 MB


In [ ]:
## Collecting smiliarty between Datasets with cosine similarity

In [469]:
# Load the data like we did before
Train = pd.read_csv('C:\\Users\Manu\\Documents\\Untitled Folder\\Orginial Dataset\\BOOKSMASTERTRAIN.csv')
Test = pd.read_csv('C:\\Users\Manu\\Documents\\Untitled Folder\\Orginial Dataset\\BOOKSMASTERTEST.csv')
Catalogue = pd.read_csv('C:\\Users\Manu\\Documents\\Untitled Folder\\Orginial Dataset\\BOOKSCATALOGUE.csv')
Train['DataSet'] = 'Train'
Test['DataSet']='Test'
BookData = pd.concat([Train,Test],axis=0).reset_index(drop=True)
del Train
del Test

In [470]:
BookData = BookData.drop(['BOOKTITLE','BOOKNAME','SERIES','AUTHOR','OTHERPRINTEDINFO','DETAILS','SUMMARY','GENRE.1','AUTHORDESC','COVERPAGE'],axis=1)


In [471]:
#New_Data['GENRE'].value_counts()
Genre_wide = pd.get_dummies(BookData['GENRE'])
BookData = pd.concat([BookData, Genre_wide], axis=1).drop(['GENRE'],axis=1) 
del Genre_wide

In [465]:
CatWide = (Catalogue['SUBSTATE'].str.split(',')
                    .groupby(Catalogue['BookID'])
                    .agg(lambda x: ', '.join(set(y for z in x for y in z)))
                    .reset_index())

CatWide_one = CatWide.SUBSTATE.str.get_dummies(sep=",")
CatWide = pd.concat([CatWide,CatWide_one],axis =1).drop('SUBSTATE',axis=1)
del CatWide_one

In [473]:
BookData = pd.merge(BookData,CatWide,left_on='BookID',right_on='BookID',how='left')

In [478]:
#from sklearn import preprocessing
# Create x, where x the 'scores' column's values as floats
x = BookData[['Popularity']].values.astype(float)

# Create a minimum and maximum processor object
min_max_scaler = MinMaxScaler()

# Create an object to transform the data to fit minmax processor
x_scaled = min_max_scaler.fit_transform(x)

# Run the normalizer on the dataframe
df_Popularity = pd.DataFrame(x_scaled,columns=['Popularity'])

In [479]:
#from sklearn import preprocessing
# Create x, where x the 'scores' column's values as floats
x = BookData[['USERRATINGS']].values.astype(float)

# Create a minimum and maximum processor object
min_max_scaler = MinMaxScaler()

# Create an object to transform the data to fit minmax processor
x_scaled = min_max_scaler.fit_transform(x)

# Run the normalizer on the dataframe
df_USERRATINGS = pd.DataFrame(x_scaled,columns=['USERRATINGS'])

In [482]:
BookData = BookData.drop(['Popularity','USERRATINGS'],axis=1)
BookData = pd.concat([BookData,df_USERRATINGS,df_Popularity],axis=1)

In [486]:
PTrain = BookData[BookData.DataSet == "Train"].fillna(0)
PTrain=PTrain.drop(['DataSet'],axis =1)
X = PTrain['BookID']
PTrain = PTrain.drop(['BookID'],axis =1)

In [492]:
PTest = BookData[BookData.DataSet == "Test"].fillna(0)
PTest=PTest.drop(['DataSet'],axis =1).reset_index(drop=True)
Y = PTest['BookID']
PTest = PTest.drop(['BookID'],axis =1)

In [495]:
from sklearn.metrics.pairwise import cosine_similarity
L = cosine_similarity(PTest,PTrain)

Matrix = pd.DataFrame(L)

# Change the column names 
Matrix.columns =X.values
  
# Change the row indexes 
Matrix.index = Y.values

##################################################

Result =[]
for i in range(len(Matrix.columns)):
    Result.append(Matrix.iloc[:, [i]].idxmax().values)

N =pd.DataFrame(Result)
X = pd.DataFrame(X)
Final_Result = pd.concat([X,N],axis =1)
Final_Result.columns = ['BookID','PURCHASEDBOOKID']

In [573]:
Final_Data['BookID'] = Final_Data['BookID'].map(Final_Result.set_index('BookID')['PURCHASEDBOOKID'])

In [576]:
Solution = Final_Data.sort_values('score', ascending=False).drop_duplicates(['UserID','BookID']).sort_index()
#Final_Data = Final_Data.drop_duplicates(["BookID","UserID"])

In [577]:
Solution.shape

(300852, 3)

In [578]:
Solution = Solution.sort_values('score',ascending = False).groupby('UserID').head(10)
Solution.shape

(108770, 3)

In [579]:
Solution = Solution.reset_index(drop = True)

In [582]:
Solution.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108770 entries, 0 to 108769
Data columns (total 2 columns):
BookID    108770 non-null float64
UserID    108770 non-null int64
dtypes: float64(1), int64(1)
memory usage: 1.7 MB


In [581]:
Solution = Solution.drop(['score'],axis=1)

In [583]:
Solution.columns = ['PURCHASEDBOOKID','USERID']

In [584]:
Final_Output = pd.DataFrame(Solution.groupby(['USERID'])['PURCHASEDBOOKID'].apply(list))

In [585]:
Final_Output.to_csv('C:\\Users\Manu\\Documents\\Untitled Folder\\final_output.csv')

In [586]:
Final_Output.shape

(10877, 1)

In [23]:
User_master = pd.read_csv('C:\\Users\Manu\\Documents\\Untitled Folder\\Orginial Dataset\\USERMASTER.csv')

In [24]:
User_master =User_master.drop(['SIGNUPDATE','DELETED'],axis =1)

In [25]:
User_master.head()

,UserID,AGEGROUP,GENDER,STATE
0,600003,28.0,FEMALE,NaN
1,600008,43.0,MALE,NaN
2,600011,35.0,MALE,NaN
3,600017,27.0,MALE,Massachusetts
4,600020,53.0,MALE,Pennsylvania


In [26]:
#from sklearn import preprocessing
# Create x, where x the 'scores' column's values as floats
x = User_master[['AGEGROUP']].values.astype(float)

# Create a minimum and maximum processor object
min_max_scaler = MinMaxScaler()

# Create an object to transform the data to fit minmax processor
x_scaled = min_max_scaler.fit_transform(x)

# Run the normalizer on the dataframe
df_AGEGROUP = pd.DataFrame(x_scaled,columns=['AGEGROUP'])
User_master = User_master.drop(['AGEGROUP'],axis=1)

In [27]:
df_AGEGROUP.head()

,AGEGROUP
0,0.200000
1,0.430769
2,0.307692
3,0.184615
4,0.584615


In [28]:
User_master['STATE'] = User_master['STATE'].fillna('Other')

In [29]:
#New_Data['GENRE'].value_counts()
Gender_state_wide = pd.get_dummies(data = User_master,columns = ['GENDER','STATE'])
Gender_state_wide.head()

,UserID,GENDER_FEMALE,GENDER_MALE,STATE_Alabama,STATE_Arizona,STATE_Arkansas,STATE_California,STATE_Colorado,STATE_Connecticut,STATE_Delaware,...,STATE_Rhode Island,STATE_South Carolina,STATE_South Dakota,STATE_Tennessee,STATE_Texas,STATE_Utah,STATE_Virginia,STATE_Washington,STATE_West Virginia,STATE_Wisconsin
0,600003,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,600008,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,600011,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,600017,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,600020,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
User_master = pd.concat([Gender_state_wide,df_AGEGROUP],axis=1)

In [31]:
User_master.head()

,UserID,GENDER_FEMALE,GENDER_MALE,STATE_Alabama,STATE_Arizona,STATE_Arkansas,STATE_California,STATE_Colorado,STATE_Connecticut,STATE_Delaware,...,STATE_South Carolina,STATE_South Dakota,STATE_Tennessee,STATE_Texas,STATE_Utah,STATE_Virginia,STATE_Washington,STATE_West Virginia,STATE_Wisconsin,AGEGROUP
0,600003,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.200000
1,600008,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.430769
2,600011,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.307692
3,600017,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.184615
4,600020,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.584615


In [32]:
known_users = pd.read_csv('C:\\Users\Manu\\Documents\\Untitled Folder\\Orginial Dataset\\BOOKSPURCHHISTORY.csv')

In [36]:
known_users_list =known_users['UserID'].tolist()

In [39]:
def search(x):
    if x in known_users_list:
        return 'Known'
    else:
        return 'UnKnown'
        

In [40]:
User_master['DataSet'] = User_master['UserID'].apply(lambda x:search(x))

In [43]:
PTrain = User_master[User_master.DataSet == "Known"].fillna(0)
PTrain=PTrain.drop(['DataSet'],axis =1)
X = PTrain['UserID']
PTrain = PTrain.drop(['UserID'],axis =1)

PTest = User_master[User_master.DataSet == "UnKnown"].fillna(0)
PTest=PTest.drop(['DataSet'],axis =1).reset_index(drop=True)
Y = PTest['UserID']
PTest = PTest.drop(['UserID'],axis =1)

In [48]:
PTest.shape

(4123, 51)

In [49]:
from sklearn.metrics.pairwise import cosine_similarity
L = cosine_similarity(PTrain,PTest)

Matrix = pd.DataFrame(L)

# Change the column names 
Matrix.columns =Y.values
  
# Change the row indexes 
Matrix.index = X.values



In [50]:
Matrix.head()

,600003,600031,600046,600079,600089,600090,600122,600146,600152,600158,...,674816,674846,674884,674917,674924,674931,674942,674964,674992,674999
600008,0.514393,0.557705,0.079304,0.553341,0.055993,0.550681,0.536398,0.521185,0.532988,0.525382,...,0.998155,0.549683,0.055993,0.103683,0.536398,0.984451,0.076468,0.992403,0.542452,0.525382
600011,0.513526,0.532286,0.057862,0.532731,0.040854,0.532242,0.526430,0.517789,0.524635,0.520318,...,0.999786,0.531981,0.040854,0.075649,0.526430,0.995478,0.055792,0.999122,0.529316,0.520318
600017,0.018126,0.502213,0.035230,0.049452,0.024875,0.509404,0.512413,0.510647,0.512305,0.511428,...,0.512305,0.509910,0.024875,0.046060,0.512413,0.507661,0.033970,0.510647,0.511997,0.511428
600020,0.053495,0.582986,0.103975,0.145947,0.526876,0.567770,0.543596,0.520741,0.538304,0.526876,...,0.538304,0.565908,0.073413,0.567770,0.543596,0.504193,0.100257,0.520741,0.553341,0.526876
600028,0.512755,0.084491,0.524459,0.526876,1.000000,0.068378,0.048544,0.032952,0.044727,0.036927,...,0.044727,0.066662,0.518464,0.985511,0.048544,0.022832,0.523913,0.032952,0.055993,0.036927


In [51]:
##################################################

Result =[]
for i in range(len(Matrix.columns)):
    Result.append(Matrix.iloc[:, [i]].idxmax().values)

N =pd.DataFrame(Result)
N.head()

,0
0,600527
1,611012
2,602156
3,600291
4,600028


In [52]:
N.shape

(4123, 1)

In [53]:
Y = pd.DataFrame(Y)
Final_Result = pd.concat([Y,N],axis =1)
Final_Result.columns = ['UnKnownUser','KnownUser']

In [54]:
User_Similarity = Final_Result.copy()

In [55]:
User_Similarity.to_csv('C:\\Users\Manu\\Documents\\Untitled Folder\\User_Similarity.csv')